In [22]:
import numpy as np
import pandas as pd
import os
import math
from nltk.tokenize import wordpunct_tokenize as tokenize


In [11]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="../resources/roberta",
    tokenizer="../resources/roberta"
)


Some weights of RobertaModel were not initialized from the model checkpoint at ./archivethird/resources and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def tokenize_sent(sequence):
    seq = sequence.replace('"', ' " ')
    seq = tokenize(seq)
    return seq

In [14]:
def continue_sent(sent):
    sent = sent + '<mask>' + '.'
    predictions = fill_mask(sent)
    return predictions[0], predictions[1]

In [132]:
def continue_with_beam(sent, topk=5, n=2):
    output_sequences = [(sent, 0)]
    
    
    for _ in range (n):
        new_sequences = []
        for sentence_longer, score in output_sequences:
            for predictions in continue_sent(sentence_longer):
                new_sent, new_score = predictions['sequence'], predictions['score']
                new_sent = new_sent.rstrip('.</s>').lstrip('<s>')
                if (tokenize_sent(new_sent)[-1]) in sentence_longer:
                    continue
                new_sequences += [(new_sent, score+math.log(new_score))]
        if (len(new_sequences) == 0): return output_sequences
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:topk]
    
    return output_sequences

In [140]:
continue_with_beam('მეტალისტი ხარკოვი ', n=10)

[('მეტალისტი ხარკოვი საქართველოში დაიბადა თბილისში, მოსკოვში',
  -9.926417230274566)]